In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv('mushrooms.csv')
for col in df.columns:
    df[col], _ = pd.factorize(df[col])
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1
2,1,1,0,2,0,2,0,0,1,1,...,0,0,0,0,0,0,0,1,1,2
3,0,0,1,2,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,3,1,3,0,1,1,0,...,0,0,0,0,0,0,1,1,2,1


In [5]:
df = df.sample(frac=1)
X = df.drop(['class'], axis=1).values
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
y = df['class'].values

split = int(X.shape[0] * .85)
X_train, y_train, X_test, y_test = X[:split], y[:split], X[split:], y[split:]

In [7]:
from keras.optimizers import Adam
from keras.layers import Dense, Flatten, Reshape
from keras.models import Sequential
from keras import callbacks
from keras.layers import Convolution1D, Dropout, BatchNormalization

model = Sequential()
model.add(Convolution1D(256, X.shape[1],  activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Reshape((256, 1)))
model.add(Convolution1D(256, X.shape[1],  activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(1, activation='relu'))
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train, y_train, batch_size=32, epochs=1000, verbose=2,
          callbacks=[callbacks.EarlyStopping(patience=200),
                     callbacks.ModelCheckpoint('1_layer_fully_conv', save_best_only=True, save_weights_only=True)],
          validation_split=0.1)

Train on 6214 samples, validate on 691 samples
Epoch 1/1000


2s - loss: 2.5916 - acc: 0.4094 - val_loss: 2.4210 - val_acc: 0.3068
Epoch 2/1000


1s - loss: 3.0622 - acc: 0.3877 - val_loss: 1.6860 - val_acc: 0.4559
Epoch 3/1000


1s - loss: 2.1108 - acc: 0.4224 - val_loss: 1.3929 - val_acc: 0.4472
Epoch 4/1000


1s - loss: 2.1332 - acc: 0.4231 - val_loss: 1.4660 - val_acc: 0.4472
Epoch 5/1000


...

1s - loss: 1.4701 - acc: 0.4079 - val_loss: 5.0757 - val_acc: 0.1650
Epoch 77/1000


1s - loss: 1.4853 - acc: 0.4042 - val_loss: 4.9142 - val_acc: 0.1751
Epoch 78/1000


1s - loss: 1.4849 - acc: 0.4034 - val_loss: 4.9142 - val_acc: 0.1751
Epoch 79/1000


1s - loss: 1.4100 - acc: 0.4057 - val_loss: 4.9388 - val_acc: 0.1722
Epoch 80/1000


1s - loss: 1.4926 - acc: 0.4030 - val_loss: 4.9604 - val_acc: 0.1722
Epoch 81/1000
